# imports


In [ ]:
import sys
import os
import psutil
import gc

import random
import numpy as np
import pandas as pd
import scipy as sp
from scipy.sparse import csr_matrix
from scipy import io

import anndata as ad
import scanpy as sc
import h5py

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sea
from pylab import rcParams

from os import listdir
from os.path import isfile, join

import copy as cp
import warnings

In [ ]:
# Toggle as needed
warnings.filterwarnings("ignore")

In [ ]:
# Set this to your source_data directory
source_data_path = ".../Source Data/"

# Functions and variables

In [ ]:
def filter_adata(cohort):
    # load data
    adata = ad.read_h5ad('%s/Single_Cell/%s_all_samples_anndata.h5ad.gzip'%(source_data_path, cohort))

    # filter out by expression 
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=50)
    sc.pl.highest_expr_genes(adata, n_top=20, )
    
    # annotate mito and ribosome genes and do qc
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    adata.var['ribo'] = adata.var_names.str.startswith('RPS') + adata.var_names.str.startswith('RPL')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo'], percent_top=None, log1p=False, inplace=True)
    sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
                 jitter=0.4, multi_panel=True)
    
    if "Samples" in adata.obs.columns:
        adata.obs["sample"] = adata.obs["Samples"]

    # plot qc
    rcParams['figure.figsize'] = 5, 5
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_mt')
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_ribo')

    sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color='sample')
    sc.pl.scatter(adata, x='total_counts', y='pct_counts_ribo', color='sample')

    # further filter data based on chosen qc metrics
    adata = adata[(adata.obs.n_genes_by_counts < 8000) & (adata.obs.n_genes_by_counts > 1000), :] # heavy filter
    adata = adata[(adata.obs.total_counts < 100000) & (adata.obs.total_counts > 3000), :]
    adata = adata[(adata.obs.pct_counts_mt < 15) & (adata.obs.pct_counts_ribo < 20), :] # heavy filter

    adata.write('%s/Single_Cell/all_%s_samples_filtered_anndata.h5ad.gzip'%(source_data_path, cohort), compression='gzip')


In [ ]:
def norm_adata(cohort, outFSuf="_all_samples_integrated_anndata"):
    # load data
    adata = ad.read_h5ad('%s/Single_Cell/all_%s_samples_filtered_anndata.h5ad.gzip'%(source_data_path, cohort))
    
    # normalize 
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    
    # highly var
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    sc.pl.highly_variable_genes(adata)
    
    # regress
    cell_cycle_genes = [x.strip() for x in open('%s/Single_Cell/regev_lab_cell_cycle_genes.txt'%(source_data_path))]
    s_genes = cell_cycle_genes[:43]
    g2m_genes = cell_cycle_genes[43:]
    adata = adata[:, adata.var.highly_variable]
    cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
    sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

    for c in ['S_score', 'G2M_score', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo']: #'total_counts'
        sc.pl.matrixplot(adata, ['LGR5', 'DLL1'], c, title=c)

    sc.pp.regress_out(adata, ['S_score', 'G2M_score'])
    sc.pp.scale(adata, max_value=10)

    # PCA after regressing out
    adata_cc_genes = adata[:, cell_cycle_genes]
    sc.tl.pca(adata_cc_genes, random_state=123)
    sc.pl.pca_scatter(adata_cc_genes, color='phase')

    
    
    random.seed(123)
    sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack', random_state=123)
    sc.pp.neighbors(adata, random_state=123)
    sc.tl.umap(adata, random_state=123)


    sc.pl.pca_scatter(adata, color= ['pct_counts_ribo', 'pct_counts_mt', 'total_counts','sample'])
    sc.pl.umap(adata, color= ['phase','total_counts','sample'])


    sc.external.pp.harmony_integrate(adata, 'sample')

    adata.obsm['X_pca'] = adata.obsm['X_pca_harmony']
    sc.pl.pca(adata, color='sample')


    random.seed(123)
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40, random_state=123)
    sc.tl.umap(adata, random_state=123)
    sc.tl.leiden(adata, resolution = 0.5, random_state=123)
    sc.pl.umap(adata, ncols = 3, color=['pct_counts_ribo', 'pct_counts_mt', 'total_counts',
                                        'phase',  'sample']) 
    sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')
    
    random.seed(123)
    sc.tl.leiden(adata, resolution = 0.9)
    sc.pl.umap(adata, color='leiden', legend_loc='on data')

    adata.write('%s/Single_Cell/%s%s.h5ad.gzip'%(source_data_path, cohort, outFSuf), compression='gzip')



In [ ]:
def get_markergenes(marker_genes, adata=None, verbose=False, subset=True, verbose_all=False):
    
    marker_genes = cp.deepcopy(marker_genes)
   
    if adata is not None:
        
        # removes ones not in adata data and made list valid subset of adata.var
        adata_valid_list = adata.var_names
        if not adata.var_names.equals(adata.var.index):
            print("!!!!!!!!anndata.var does not match anndata.var_names!!!!!!!!")
        
        if isinstance(marker_genes, dict): 
            empty_keys = []
            for group_key in marker_genes:
                to_remove = []
                if verbose:
                    print("############### Current group is %s"%(group_key))
                for gene in marker_genes[group_key]:
                    if verbose_all:
                        print("%s not in adata_valid_list = %s"%(gene, gene not in adata_valid_list))
                    if gene not in adata_valid_list:
                        to_remove.append(gene)

                for gene in to_remove:
                    if verbose:
                            print("removing %s"%(gene))
                    marker_genes[group_key].remove(gene)
                if marker_genes[group_key] == []:
                    empty_keys.append(group_key)

            for group_key in empty_keys:
                del marker_genes[group_key]
        elif isinstance(marker_genes, list) :
            to_remove = []
            for gene in marker_genes:
                if verbose_all:
                    print("%s not in adata_valid_list = %s"%(gene, gene not in adata_valid_list))
                if gene not in adata_valid_list:
                        to_remove.append(gene)
            for gene in to_remove:
                if verbose:
                    print("removing %s"%(gene))
                marker_genes.remove(gene)
        elif isinstance(marker_genes, pd.core.series.Series):
            to_remove = []
            for gene in marker_genes:
                if verbose_all:
                    print("%s not in adata_valid_list = %s"%(gene, gene not in adata_valid_list))
                if gene not in adata_valid_list:
                        to_remove.append(marker_genes.index[marker_genes==gene].tolist()[0])
            if verbose:
                print(to_remove)
            for index in to_remove:
                if verbose:
                    print("removing %s at index %s"%(marker_genes[index], index))
                marker_genes.drop(index, inplace=True)
        elif verbose:
            print("Not valid type for subsetting")


    return marker_genes


In [ ]:
def plot_dotplot(adata, gene_list_list, group_by='leiden', title_name="", verbose=False, subset=True, verbose_all=False):
    for gene_list in gene_list_list:
        if subset:
            gene_list = get_markergenes(gene_list, adata, verbose=verbose, verbose_all= verbose_all, subset=subset)
        sc.pl.dotplot(adata, gene_list, group_by, title=title_name)
        

In [ ]:
all_marker_genes = {}

all_marker_genes['Cancer subtypes'] =  ['TFF1', 'FGA', 'CPS1', 'NKX2-1', 'SFTA3', 'SFTPC', 'KRT5', 'KRT6B', 'TP63', 'NCAM1',   'CHGA',   'CHGB']



all_marker_genes['Natural killer cells'] = ['KLRC1' ,'KLRD1' ,'NKG7']
all_marker_genes['T cells'] = ['CD2', 'CD3D', 'CD3E', 'CD3G']
all_marker_genes['CD8+ T cells'] = ['NKG7', 'CD8A', 'GNLY', 'GZMA', 'GZMB', 'GZMK', 'GZMH']
all_marker_genes['CD4+ T cells'] = ['CD4']
all_marker_genes['Naïve T cells'] = ['CCR7' ,'LEF1' ,'IL7R' ,'SELL']
all_marker_genes['Exhausted T cells'] = ['LAG3' , 'TIGIT', 'CTLA4']
all_marker_genes['Regulator T cells'] = ['FOXP3' ,'IL2RA', 'IKZF2', 'CTLA4']
all_marker_genes['Tissue-resident memory T cells'] = ['ITGAE' ,'ITGA1' ,'ZNF683']

all_marker_genes['B cells'] = ['CD79A', 'CD79B']
all_marker_genes['Follicular B cells'] = ['MS4A1', 'HLA-DRB1','HLA-DRA','HLA-DRB5','HLA-DRB6', 
                                          'HLA-DRB9 ', 'CXCR4']

all_marker_genes['Plasma cells'] = ['MZB1' ,'JCHAIN' ,'IGHG1']

all_marker_genes['Myeloid cells'] = ['LYZ']

all_marker_genes['Neutrophils'] = ['CSF3R' ,'S100A8', 'S100A9' ,'FCGR3B']

all_marker_genes['cDC1'] = ['XCR1' , 'CLEC9A']
all_marker_genes['cDC2'] = ['FCER1A', 'CD1C']
all_marker_genes['Mature dendritic cells'] = ['LAMP3']
all_marker_genes['Follicular dendritic cells'] = ['FDCSP']
all_marker_genes['Plasmacytoid dendritic cells'] = ['IL3RA' ,'LILRA4' ,'CLEC4C']

all_marker_genes['Macrophages'] = ['CD68']
all_marker_genes['M2 macrophages'] = ['MRC1' ,'CD163']

all_marker_genes['Monocytes'] = ['CD14' ,'FCN1']

all_marker_genes['Mast cells'] = ['TPSAB1' ,'TPSB2','GATA2']

all_marker_genes['Tip cells'] = ['DLL4' ,'KCNE3' ,'ESM1','ANGPT2']

all_marker_genes['Endothelial cells'] = ['CLDN5' ,'PECAM1' ,'VWF']
all_marker_genes['Vein endothelial cells'] = ['ACKR1']
all_marker_genes['Artery endothelial cells'] = ['GJA5']
all_marker_genes['Lymphatic endothelial cells'] = ['PROX1' ,'PDPN']

all_marker_genes['Pericytes'] = ['RGS5' ,'CSPG4']

all_marker_genes['Fibroblasts'] = ['DCN' ,'COL1A1', 'COL1A2' ]
all_marker_genes['Myofibroblasts'] = ['ACTA2' ,'MYH11']

all_marker_genes['Epithelial cells'] = ['CAPS' ,'SNTN'] 

all_marker_genes['Alveolar cells'] = ['CLDN18' ,'AQP4'] 
all_marker_genes['Alveolar type 1 cells'] = ['CAV1' ,'AGER'] 
all_marker_genes['Alveolar type 2 cells'] = ['SFTPC' , 'SFTPA1' , 'ABCA3'] 

all_marker_genes['Club cells'] = ['SCGB1A1' ,'SCGB3A1'] 

all_marker_genes['Basal cells'] = ['KRT5' ,'KRT6A' ,'KRT14'] 

all_marker_genes['Ciliated cells'] = ['FOXJ1' ,'TPPP3' ,'PIFO'] 

all_marker_genes['Cancer drivers'] = ['EGFR', 'ALK','RET', 'HER2'] 

In [ ]:
sub_marker_genes = {}
sub_marker_genes['Cancer'] = ['EGFR', 'ALK','RET', 'HER2',
                             'EPCAM', 
                              'TFF1', 'NKX2-1', 'KRT5', 'NCAM1', 'FGA', 'SFTA3', 'KRT6B', 'CHGA', 'CPS1', 'SFTPC', 'TP63', 'CHGB'

                             ]
sub_marker_genes['Myeloid'] = ['LYZ',
                               'CD14', 'CD68', 'FCGR1A', 'FCGR3A', 'MNDA' 
                              ]
sub_marker_genes['Fibroblast'] = ['DCN' ,'COL1A1', 'COL1A2', 'ACTA2' ,'MYH11',
                                  'LUM', 'C1R' 
                                 ]
sub_marker_genes['T_cell'] = ['CD2', 'CD3D', 'CD3E', 'CD3G', 'NKG7', 'CD8A', 'GNLY', 'GZMA', 'GZMB', 'GZMK', 'GZMH', 'CD4', 
                              'CCR7' ,'LEF1' ,'IL7R' ,'SELL', 'LAG3' , 'TIGIT', 'CTLA4', 'FOXP3' ,'IL2RA', 'IKZF2', 'CTLA4',
                              'ITGAE' ,'ITGA1' ,'ZNF683',
                              'TRAC' 
                             ]
sub_marker_genes['B_cell'] = ['CD79A', 'CD79B', 'MS4A1', 'HLA-DRB1', 'HLA-DRA', 'HLA-DRB5', 'HLA-DRB6', 'HLA-DRB9 ', 'CXCR4',
                              'MZB1' ,'JCHAIN' ,'IGHG1',
                              'IGLC3' 
                             ]
sub_marker_genes['Neutrophil'] = ['CSF3R' ,'S100A8', 'S100A9' ,'FCGR3B',
                                  'CXCR2', 'FCGR3B' 
                                 ]
sub_marker_genes['Alveolar'] = ['CLDN18' ,'AQP4', 'CAV1' ,'AGER', 'SFTPC' , 'SFTPA1' , 'ABCA3',
                                'FOLR1', 'SFTPB', 'SCGB3A1' 
                               ]
sub_marker_genes['Epithelial'] = ['CAPS' ,'SNTN',
                                  'TFF3', 'CDH1', 'FOXJ1' 
                                 ]
sub_marker_genes['Endothelial'] = ['CLDN5' , 'PECAM1' , 'VWF', 'ACKR1', 'GJA5', 'PROX1' ,'PDPN',
                                   'FLT1', 'KDR', 'CDH5', 'ANGPT2' 
                                  ]
sub_marker_genes['Mast_cell'] = ['TPSAB1' ,'TPSB2','GATA2',
                                 'CPA3', 'MS4A2' 
                                ]
sub_marker_genes['fDC'] = ['FDCSP']



# set cohort and load starting data

In [ ]:
# Wu data
cohort = "Wu"
mypath = '%s/Single_Cell/raw_data_exp_matrix'%(source_data_path)
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
all_samples = [f.split('_')[1] for f in listdir(mypath) if isfile(join(mypath, f))]

all_adata = [sc.read('%s/%s'%(mypath, dir_path), cache=True, deliminator = '\t').transpose() for dir_path in onlyfiles]

for i in range(len(all_samples)):
    sample_col = pd.DataFrame({'sample':[all_samples[i]]*(all_adata[i].obs.shape[0]), 'idx':all_adata[i].obs.index.tolist()}).set_index('idx')
    sample_col.index.name = None
    all_adata[i].obs = (all_adata[i].obs).join(sample_col)

In [ ]:
merged_all_adata = all_adata[0].concatenate(*all_adata[1:len(all_adata)], batch_key='sample_n')
merged_all_adata.var_names_make_unique() 
adata = merged_all_adata


In [ ]:
adata.write('%s/Single_Cell/%s_all_samples_anndata.h5ad.gzip'%(source_data_path, cohort), compression='gzip')

## filter and normalize Wu data

In [ ]:
filter_adata(cohort)

In [ ]:
norm_adata(cohort, outFSuf="_all_samples_integrated_anndata_paper")

# Label clusters 

In [ ]:
adata  = ad.read('%s/Single_Cell/Wu_all_samples_integrated_anndata_paper_lock.h5ad.gzip'%(source_data_path))


# cell type dotplots

In [ ]:
# determine cluster cell type
plot_dotplot(adata,  [sub_marker_genes, all_marker_genes], title_name = "Wu cell type leiden" )

## Cell type annotations

In [ ]:
leiden_clust_annotation_min = {
    '0' : 'Cancer-Squamous',
    '1' : 'Cancer-Adeno/Squamous',
    '2' : 'Myeloid',
    '3' : 'Cancer-Squamous',
    '4' : 'Fibroblasts',
    '5' : 'Myeloid', 
    '6' : 'Cancer-Squamous',
    '7' : 'Cancer-Squamous',
    '8' : 'Cancer-Squamous',
    '9' : 'Myeloid', 
    '10': 'Cancer-Adeno', 
    '11': 'Cancer-Squamous',
    '12': 'Cancer-DeDifferentiated', 
    '13': 'B cells', 
    '14': 'Fibroblasts',
    '15': 'T cell',
    '16': 'Cancer-Squamous', 
    '17': 'Endothelial', 
    '18': 'Alveolar',
    '19': 'Epithelial',
    '20': 'Myeloid', 
    '21': 'Cancer-Adeno',
    '22': 'Cancer-Adeno/Squamous', 
    '23': 'Cancer-Squamous',
    '24': 'T cell',
    '25': 'Myeloid',
}

## leiden labeling

In [ ]:
adata.obs['leiden_annotation_min'] = [leiden_clust_annotation_min[l] for l in adata.obs['leiden']]

adata = adata.raw.to_adata()
for cell in adata.obs.index:
    l = adata.obs.loc[cell, 'leiden']
    cell_type = adata.obs.loc[cell, 'leiden_annotation_min']
    adata.obs.loc[cell, 'leiden_annotation_pair'] = '%s-%s'%(cell_type, l)
    


In [ ]:
adata.write('%s/Single_Cell/Wu_integrated_leiden_ann_adata.h5ad.gzip'%(source_data_path), compression="gzip")